In [22]:
import requests
import io
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
from typing import Tuple, List
import re
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from statistics import mode
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [23]:
def print_tabulate(df: pd.DataFrame):
    print(tabulate(df, headers=df.columns, tablefmt='orgtbl'))

def classification(row, col):
    if row[col] < 150:
        return 1
    elif row[col] < 200:
        return 2
    return 3

def euclidean_distance(p_1, p_2)->float:
    return np.sqrt(np.sum((p_2 - p_1) ** 2))

def scatter_group_by(file_path: str, df: pd.DataFrame, x_column: str, y_column: str, label_column: str):
    colors = ["blue", "gray", "red"]
    fig, ax = plt.subplots()
    labels = pd.unique(df[label_column])
    
    for i, label in enumerate(labels):
        df[df[label_column] == label]
        filter_df = df[df[label_column] == label]
        ax.scatter(filter_df[x_column], filter_df[y_column], label=label, color=colors[i])

    plt.title("Sales per flight")
    plt.xlabel("Flight")
    plt.ylabel("Sales")
    ax.legend()
    plt.savefig(file_path)
    plt.close()

def k_nearest_neighbors(points, labels, input_data, k):
    input_distances = [
        [euclidean_distance(input_point, point) for point in points]
        for input_point in input_data
    ]
    points_k_nearest = [
        np.argsort(input_point_dist)[:k] for input_point_dist in input_distances
    ]
    return [
        mode([labels[index] for index in point_nearest])
        for point_nearest in points_k_nearest
    ]

In [24]:
data = pd.read_csv('flight_sales.csv')
print_tabulate(data.head())

|    |   Unnamed: 0 | Flight_ID                        | ProductType   | ProductName    |   Quantity |   TotalSales | Aeronave   | DepartureStation   | ArrivalStation   | Destination_Type   | Origin_Type         | STD                 | STA                 |   Capacity |   Passengers |   Bookings |
|----+--------------+----------------------------------+---------------+----------------+------------+--------------+------------+--------------------+------------------+--------------------+---------------------+---------------------+---------------------+------------+--------------+------------|
|  0 |            0 | 00004a718edba9d9ef878d08f02ae057 | Botanas       | Luxury Nut Mix |          1 |           55 | XA-VYD     | AL                 | AT               | Ciudad Principal   | MX Amigos y Familia | 2023-12-18 16:20:00 | 2023-12-18 18:35:00 |        180 |          174 |        106 |
|  1 |            1 | 00004a718edba9d9ef878d08f02ae057 | Licores       | Xx Lager       |          1 | 

In [25]:
data.insert(6,"TotalSalesPerFlight", data.groupby('Flight_ID', as_index=False)['TotalSales'].sum().dropna()['TotalSales'])
# print(data.groupby('Flight_ID')['TotalSales'].sum().head())
print_tabulate(data.head())

|    |   Unnamed: 0 | Flight_ID                        | ProductType   | ProductName    |   Quantity |   TotalSales |   TotalSalesPerFlight | Aeronave   | DepartureStation   | ArrivalStation   | Destination_Type   | Origin_Type         | STD                 | STA                 |   Capacity |   Passengers |   Bookings |
|----+--------------+----------------------------------+---------------+----------------+------------+--------------+-----------------------+------------+--------------------+------------------+--------------------+---------------------+---------------------+---------------------+------------+--------------+------------|
|  0 |            0 | 00004a718edba9d9ef878d08f02ae057 | Botanas       | Luxury Nut Mix |          1 |           55 |                  2028 | XA-VYD     | AL                 | AT               | Ciudad Principal   | MX Amigos y Familia | 2023-12-18 16:20:00 | 2023-12-18 18:35:00 |        180 |          174 |        106 |
|  1 |            1 | 00004a718

In [26]:
dataFlights = data.groupby(["Flight_ID"], as_index=False)['TotalSalesPerFlight'].sum()
print_tabulate(dataFlights.head())
dataFlights.plot("Flight_ID", "TotalSalesPerFlight", color='orange', marker='o', title='Sales per Flight', legend=False, grid=True)
plt.xlabel("Flight")
plt.ylabel("Total Sales")
plt.savefig("img/Line.png")
plt.close()

|    | Flight_ID                        |   TotalSalesPerFlight |
|----+----------------------------------+-----------------------|
|  0 | 00004a718edba9d9ef878d08f02ae057 |                 26207 |
|  1 | 0000a70ace2593137b79ace332478392 |                     0 |
|  2 | 0000c5ba279c7225e9f6bac8490678e1 |                     0 |
|  3 | 0000cd79c0c3a9c309df6064dcacaeea |                 17192 |
|  4 | 00010d4c88e2cb6089937fd4e4f9783b |                     0 |


In [27]:
print_tabulate(data.describe())

|       |       Unnamed: 0 |     Quantity |      TotalSales |   TotalSalesPerFlight |      Capacity |    Passengers |      Bookings |
|-------+------------------+--------------+-----------------+-----------------------+---------------+---------------+---------------|
| count |      2.78422e+06 |  2.64494e+06 |     2.64494e+06 |             242630    |   2.69919e+06 |   2.57563e+06 |   2.57563e+06 |
| mean  |      1.39211e+06 |  2.55676     |   112.65        |               1228.01 | 209.853       | 188.493       | 113.064       |
| std   | 803736           |  2.92035     |   177.483       |               2325.13 |  25.6721      |  35.5827      |  29.1993      |
| min   |      0           |  1           | -3234           |                -14    | 178           |   1           |   1           |
| 25%   | 696056           |  1           |    45           |                  0    | 186           | 167           |  92           |
| 50%   |      1.39211e+06 |  2           |    80           | 

In [28]:
dataFlights['Grupo'] = dataFlights.apply(lambda row: classification(row, 'TotalSalesPerFlight'), axis=1)

scatter_group_by("img/groups.png", dataFlights, "Flight_ID", "TotalSalesPerFlight", "Grupo")


dataFlights.head()

KeyboardInterrupt: 